In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)
payroll.head()

In [ ]:
payroll.fillna(0, inplace=True)

In [ ]:
payroll.drop(['First Name', 'Last Name', 'Middle Init', 'Bureau', 'Office', 'Office Name', 'Job Title'],axis=1,inplace=True)
payroll.head()

In [ ]:
pr = payroll.loc[payroll['Job Code'] == 1172]
pr

In [ ]:
len(pr.axes[0])
pr.info()

In [ ]:
_index = payroll['Job Code'].unique()
_ids = []

try:
  with open('../Datasets/ids.txt', "r") as f:
    arr =[]
    for line in f:
       arr.append(int(line))
    print(arr)
except:
  for _id in _index:
      counts = len(payroll.groupby(['Job Code']).get_group(_id))
      if counts > 1000:
          sid = str(_id)
          _ids.append(sid)
          
  with open('../Datasets/ids.txt', 'w') as f:
    for _id in _ids:
      f.write('%s\n' % _id)
finally:
   f.close()


In [ ]:
pr = pr.sort_values(by=['Employee Identifier'], ascending=True)

In [ ]:
fiscal = pd.pivot_table(pr, values='Base Pay', index=['Fiscal Period'], columns='Employee Identifier')
fiscal

In [ ]:
fiscal.diff()